In [1]:
import os
import logging
import pandas as pd
import numpy as np
from questions import answer_question
from telegram import Update
from telegram.ext import ContextTypes, CommandHandler, ApplicationBuilder
from openai import OpenAI
import asyncio
import nest_asyncio

In [2]:
nest_asyncio.apply()

In [3]:
df = pd.read_csv('processed/embeddings.csv', index_col=0)
df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)

In [4]:
openai = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
tg_bot_token = os.environ['TG_BOT_TOKEN']

In [5]:
messages = [{
    "role":"system",
    "content":"You are a helpful assistant that answers questions."
}]

In [6]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - $(levelname)s - %(message)s',
    level=logging.INFO
)

In [7]:
async def chat(update: Update, context: ContextTypes.DEFAULT_TYPE):
    messages.append({"role":"user","content":update.message.text})
    completion = openai.chat.completions.create(model="gpt-3.5-turbo",
                                               messages=messages)
    completion_answer=completion.choices[0].message
    messages.append(completion_answer)

    await context.bot.send_message(chat_id=update.effective_chat.id,
                                  text=completion_answer.content)

In [8]:
async def mozilla(update: Update, context: ContextTypes.DEFAULT_TYPE):
    answer = answer_question(df, question=update.message.text, debug=True)
    await context.bot.send_message(chat_id=update.effective_chat.id, text=answer)

In [9]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id,
                                  text="I am a bot, please talk to me.")

In [10]:
# Main function to run the bot
async def main() -> None:
    application = ApplicationBuilder().token(tg_bot_token).build()

    start_handler = CommandHandler('start', start)
    chat_handler = CommandHandler('chat', chat)
    mozilla_handler = CommandHandler('mozilla', mozilla)

    application.add_handler(start_handler)
    application.add_handler(chat_handler)
    application.add_handler(mozilla_handler)

    await application.run_polling()

In [ ]:
# Check if the script is run directly (not imported)
if __name__ == '__main__':
    try:
        asyncio.run(main())
    except RuntimeError as e:
        if str(e) == "asyncio.run() cannot be called from a running event loop":
            loop = asyncio.get_event_loop()
            loop.run_until_complete(main())
        else:
            raise

2024-07-03 15:17:58,480 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getMe "HTTP/1.1 200 OK"
2024-07-03 15:17:58,672 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/deleteWebhook "HTTP/1.1 200 OK"
2024-07-03 15:17:58,674 - apscheduler.scheduler - $(levelname)s - Scheduler started
2024-07-03 15:17:58,675 - telegram.ext.Application - $(levelname)s - Application started
